# Orderbook Implementation - Complete Solution

## Overview
An optimal orderbook using:
- **Heaps** (heapq) - Track best bid/ask prices in O(log P)
- **Deques** (collections.deque) - FIFO queues in O(1)
- **Dictionaries** - Fast O(1) order lookups

## Why Negative Prices for Bids?
Python's `heapq` only implements **min heaps**. To get max heap behavior for bids:
- Store negative prices: -100, -95, -90
- Min heap puts -100 at top (smallest)
- Negate to get actual: -(-100) = $100 (highest) ✓

## Imports

In [ ]:
from collections import deque
import heapq
from enum import Enum
from typing import List, Tuple, Optional, Dict

## Enums

In [ ]:
class Side(Enum):
    BUY = "BUY"
    SELL = "SELL"

class OrderStatus(Enum):
    OPEN = "OPEN"
    PARTIALLY_FILLED = "PARTIALLY_FILLED"
    FILLED = "FILLED"
    CANCELLED = "CANCELLED"

## Order Class

In [ ]:
class Order:
    """Represents a single order in the orderbook."""

    def __init__(self, order_id: str, side: Side, price: float, quantity: int, timestamp: int):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.quantity = quantity
        self.filled_quantity = 0
        self.status = OrderStatus.OPEN
        self.timestamp = timestamp

    def remaining_quantity(self) -> int:
        """Returns the unfilled quantity of the order."""
        return self.quantity - self.filled_quantity

    def __repr__(self):
        return (f"Order(id={self.order_id}, side={self.side.value}, "
                f"price={self.price}, qty={self.quantity}, "
                f"filled={self.filled_quantity}, status={self.status.value})")

## OrderBook Class - Initialization

In [ ]:
class OrderBook:
    """
    An optimal orderbook implementation.
    
    Time Complexities:
    - Submit: O(log P)
    - Cancel: O(M)
    - Modify: O(M + log P)
    - Status: O(1)
    - Match: O(log P + K×M + K×log P)
    - Get Best: Amortized O(1)
    """

    def __init__(self):
        """Initialize the orderbook data structures."""
        # Price level management: price -> deque of orders at that price
        self.bids: Dict[float, deque] = {}  # Buy orders
        self.asks: Dict[float, deque] = {}  # Sell orders

        # Heaps to track best prices
        # For bids: use negative prices to simulate max heap (highest price first)
        # For asks: use positive prices for min heap (lowest price first)
        self.bid_heap: List[float] = []
        self.ask_heap: List[float] = []

        # Fast O(1) order lookup by order_id
        self.orders: Dict[str, Order] = {}

        # Timestamp counter for order sequencing (ensures FIFO at same price)
        self.timestamp = 0

## Submit Order - O(log P)

In [ ]:
def submit_order(self, order_id: str, side: Side, price: float, quantity: int) -> bool:
    """
    Submit a new order to the orderbook.
    
    Time Complexity: O(log P) where P is the number of unique price levels
    """
    # Check if order already exists
    if order_id in self.orders:
        return False

    # Create new order with timestamp
    self.timestamp += 1
    order = Order(order_id, side, price, quantity, self.timestamp)
    self.orders[order_id] = order

    # Add to appropriate side
    if side == Side.BUY:
        # Add to bids
        if price not in self.bids:
            self.bids[price] = deque()
            # Use negative price for max heap behavior
            heapq.heappush(self.bid_heap, -price)
        self.bids[price].append(order)
    else:  # Side.SELL
        # Add to asks
        if price not in self.asks:
            self.asks[price] = deque()
            # Use positive price for min heap behavior
            heapq.heappush(self.ask_heap, price)
        self.asks[price].append(order)

    return True

# Add method to OrderBook class
OrderBook.submit_order = submit_order

## Cancel Order - O(M)

In [ ]:
def cancel_order(self, order_id: str) -> bool:
    """
    Cancel an existing order.
    
    Time Complexity: O(M) where M is orders at that price level
    """
    # Look up order
    if order_id not in self.orders:
        return False

    order = self.orders[order_id]

    # Check if order can be cancelled
    if order.status in [OrderStatus.FILLED, OrderStatus.CANCELLED]:
        return False

    # Mark as cancelled
    order.status = OrderStatus.CANCELLED

    # Remove from price level queue
    price_levels = self.bids if order.side == Side.BUY else self.asks
    if order.price in price_levels:
        queue = price_levels[order.price]
        # Need to scan and remove (O(M) where M is queue length)
        try:
            queue.remove(order)
        except ValueError:
            pass  # Order already removed

        # Clean up empty price level
        if len(queue) == 0:
            del price_levels[order.price]
            # Note: We don't remove from heap (inefficient)
            # Instead, we'll handle stale prices when accessing heap top

    return True

# Add method to OrderBook class
OrderBook.cancel_order = cancel_order

## Get Order Status - O(1)

In [ ]:
def get_order_status(self, order_id: str) -> Optional[Order]:
    """
    Get the current status of an order.
    
    Time Complexity: O(1) - direct dictionary lookup
    """
    return self.orders.get(order_id)

# Add method to OrderBook class
OrderBook.get_order_status = get_order_status

## Modify Order - O(M + log P)

In [ ]:
def modify_order(self, order_id: str, new_price: Optional[float] = None,
                 new_quantity: Optional[int] = None) -> bool:
    """
    Modify an existing order (loses time priority).
    
    Time Complexity: O(M + log P) = cancel + submit
    """
    # Look up existing order
    if order_id not in self.orders:
        return False

    order = self.orders[order_id]

    # Can't modify filled or cancelled orders
    if order.status in [OrderStatus.FILLED, OrderStatus.CANCELLED]:
        return False

    # Determine new parameters
    final_price = new_price if new_price is not None else order.price
    final_quantity = new_quantity if new_quantity is not None else order.quantity

    # New quantity must be at least the filled quantity
    if final_quantity < order.filled_quantity:
        return False

    # Cancel existing order (removes from queue)
    side = order.side
    filled_qty = order.filled_quantity
    self.cancel_order(order_id)

    # Submit new order with updated parameters
    success = self.submit_order(order_id, side, final_price, final_quantity)

    if success and filled_qty > 0:
        # Preserve filled quantity if there was any
        self.orders[order_id].filled_quantity = filled_qty
        self.orders[order_id].status = OrderStatus.PARTIALLY_FILLED if filled_qty < final_quantity else OrderStatus.FILLED

    return success

# Add method to OrderBook class
OrderBook.modify_order = modify_order

## Match Order - O(log P + K×M + K×log P)

In [ ]:
def match_order(self, order_id: str, side: Side, price: float, quantity: int) -> List[Tuple[str, str, float, int]]:
    """
    Submit an order and match it against the opposite side of the book.
    
    Time Complexity: O(log P + K×M + K×log P)
    where P = price levels, K = matched levels, M = orders per level
    """
    trades = []

    # Submit the order first
    if not self.submit_order(order_id, side, price, quantity):
        return trades  # Order already exists

    incoming_order = self.orders[order_id]

    # Determine opposite side
    if side == Side.BUY:
        # Match buy order against asks
        opposite_heap = self.ask_heap
        opposite_levels = self.asks
        is_buy = True
    else:
        # Match sell order against bids
        opposite_heap = self.bid_heap
        opposite_levels = self.bids
        is_buy = False

    # Keep matching while there's quantity remaining and matchable prices exist
    while incoming_order.remaining_quantity() > 0 and opposite_heap:
        # Get best opposite price
        if is_buy:
            best_price = opposite_heap[0]  # Ask: already positive (min heap)
        else:
            best_price = -opposite_heap[0]  # Bid: negate to get actual price

        # Check if price is matchable
        if is_buy and best_price > price:
            break  # Ask too high for our buy order
        if not is_buy and best_price < price:
            break  # Bid too low for our sell order

        # Clean up stale prices (prices with no orders)
        heap_price = opposite_heap[0] if is_buy else -opposite_heap[0]
        if heap_price not in opposite_levels or len(opposite_levels[heap_price]) == 0:
            heapq.heappop(opposite_heap)
            if heap_price in opposite_levels:
                del opposite_levels[heap_price]
            continue

        # Get queue at this price level
        queue = opposite_levels[best_price]

        # Match against orders in FIFO order
        while queue and incoming_order.remaining_quantity() > 0:
            resting_order = queue[0]

            # Skip cancelled/filled orders
            if resting_order.status in [OrderStatus.CANCELLED, OrderStatus.FILLED]:
                queue.popleft()
                continue

            # Calculate trade quantity
            trade_qty = min(incoming_order.remaining_quantity(), resting_order.remaining_quantity())
            trade_price = resting_order.price  # Resting order's price (price-time priority)

            # Execute trade
            incoming_order.filled_quantity += trade_qty
            resting_order.filled_quantity += trade_qty

            # Update statuses
            if incoming_order.remaining_quantity() == 0:
                incoming_order.status = OrderStatus.FILLED
            else:
                incoming_order.status = OrderStatus.PARTIALLY_FILLED

            if resting_order.remaining_quantity() == 0:
                resting_order.status = OrderStatus.FILLED
                queue.popleft()  # Remove fully filled order
            else:
                resting_order.status = OrderStatus.PARTIALLY_FILLED

            # Record trade
            if is_buy:
                trades.append((incoming_order.order_id, resting_order.order_id, trade_price, trade_qty))
            else:
                trades.append((resting_order.order_id, incoming_order.order_id, trade_price, trade_qty))

        # Clean up empty price level
        if len(queue) == 0:
            del opposite_levels[best_price]
            heapq.heappop(opposite_heap)

    return trades

# Add method to OrderBook class
OrderBook.match_order = match_order

## Get Best Bid - Amortized O(1)

In [ ]:
def get_best_bid(self) -> Optional[float]:
    """
    Get the best (highest) bid price.
    
    Time Complexity: Amortized O(1), worst case O(P)
    """
    # Clean up stale prices
    while self.bid_heap:
        price = -self.bid_heap[0]  # Negate to get actual price
        if price in self.bids and len(self.bids[price]) > 0:
            return price
        heapq.heappop(self.bid_heap)
        if price in self.bids:
            del self.bids[price]
    return None

# Add method to OrderBook class
OrderBook.get_best_bid = get_best_bid

## Get Best Ask - Amortized O(1)

In [ ]:
def get_best_ask(self) -> Optional[float]:
    """
    Get the best (lowest) ask price.
    
    Time Complexity: Amortized O(1), worst case O(P)
    """
    # Clean up stale prices
    while self.ask_heap:
        price = self.ask_heap[0]  # Already positive
        if price in self.asks and len(self.asks[price]) > 0:
            return price
        heapq.heappop(self.ask_heap)
        if price in self.asks:
            del self.asks[price]
    return None

# Add method to OrderBook class
OrderBook.get_best_ask = get_best_ask

## Get Spread - Amortized O(1)

In [ ]:
def get_spread(self) -> Optional[float]:
    """
    Get the bid-ask spread.
    
    Time Complexity: Amortized O(1)
    """
    best_bid = self.get_best_bid()
    best_ask = self.get_best_ask()

    if best_bid is None or best_ask is None:
        return None

    return best_ask - best_bid

# Add method to OrderBook class
OrderBook.get_spread = get_spread

## Pretty Print

In [ ]:
def __repr__(self):
    """Pretty print the orderbook state."""
    lines = ["=" * 60]
    lines.append("ORDER BOOK")
    lines.append("=" * 60)

    # Get all ask prices sorted high to low
    ask_prices = sorted(self.asks.keys(), reverse=True)
    for price in ask_prices:
        total_qty = sum(o.remaining_quantity() for o in self.asks[price]
                      if o.status not in [OrderStatus.CANCELLED, OrderStatus.FILLED])
        if total_qty > 0:
            lines.append(f"ASK: {price:>10.2f} | Qty: {total_qty:<10}")

    lines.append("-" * 60)

    # Get all bid prices sorted high to low
    bid_prices = sorted(self.bids.keys(), reverse=True)
    for price in bid_prices:
        total_qty = sum(o.remaining_quantity() for o in self.bids[price]
                      if o.status not in [OrderStatus.CANCELLED, OrderStatus.FILLED])
        if total_qty > 0:
            lines.append(f"BID: {price:>10.2f} | Qty: {total_qty:<10}")

    lines.append("=" * 60)

    # Show spread
    spread = self.get_spread()
    if spread is not None:
        lines.append(f"Spread: {spread:.2f}")

    return "\n".join(lines)

# Add method to OrderBook class
OrderBook.__repr__ = __repr__

## Test Suite

Run all tests to verify the implementation!

In [ ]:
# Test 1: Submit orders
print("Test 1: Submitting orders...")
ob = OrderBook()

ob.submit_order("B1", Side.BUY, 100.0, 50)
ob.submit_order("B2", Side.BUY, 99.5, 30)
ob.submit_order("B3", Side.BUY, 99.0, 20)
ob.submit_order("A1", Side.SELL, 101.0, 40)
ob.submit_order("A2", Side.SELL, 101.5, 25)
ob.submit_order("A3", Side.SELL, 102.0, 35)

print(ob)
print(f"\nBest Bid: {ob.get_best_bid()}")
print(f"Best Ask: {ob.get_best_ask()}")
print(f"Spread: {ob.get_spread()}")

In [ ]:
# Test 2: Order status
print("Test 2: Checking order status...")
order = ob.get_order_status("B1")
print(f"Order B1: {order}")

In [ ]:
# Test 3: Cancel order
print("Test 3: Cancelling order B2...")
ob.cancel_order("B2")
print(ob)

In [ ]:
# Test 4: Modify order
print("Test 4: Modifying order B3 price to 100.5...")
ob.modify_order("B3", new_price=100.5)
print(ob)

In [ ]:
# Test 5: Match order
print("Test 5: Matching a sell order at 99.5 for 60 units...")
trades = ob.match_order("S1", Side.SELL, 99.5, 60)
print(f"Trades executed: {len(trades)}")
for trade in trades:
    print(f"  Buyer: {trade[0]}, Seller: {trade[1]}, Price: {trade[2]}, Qty: {trade[3]}")
print(ob)

In [ ]:
# Test 6: Match order crossing multiple price levels
print("Test 6: Matching a buy order at 102.0 for 100 units...")
trades = ob.match_order("B4", Side.BUY, 102.0, 100)
print(f"Trades executed: {len(trades)}")
for trade in trades:
    print(f"  Buyer: {trade[0]}, Seller: {trade[1]}, Price: {trade[2]}, Qty: {trade[3]}")
print(ob)

In [ ]:
# Test 7: Check final order statuses
print("Test 7: Final order statuses:")
for order_id in ["B1", "A1", "A2", "S1", "B4"]:
    order = ob.get_order_status(order_id)
    if order:
        print(f"  {order}")

## Performance Analysis

Let's verify the time complexities with some benchmarks.

In [ ]:
import time

# Benchmark submit order
ob_bench = OrderBook()
n = 10000

start = time.time()
for i in range(n):
    ob_bench.submit_order(f"B{i}", Side.BUY, 100.0 + i * 0.01, 100)
end = time.time()

print(f"Submitted {n} orders in {(end-start)*1000:.2f}ms")
print(f"Average per order: {(end-start)/n*1e6:.2f} microseconds")

In [ ]:
# Benchmark get_best operations
start = time.time()
for _ in range(10000):
    ob_bench.get_best_bid()
end = time.time()

print(f"10,000 get_best_bid calls in {(end-start)*1000:.2f}ms")
print(f"Average per call: {(end-start)/10000*1e6:.2f} microseconds")